In [1]:
import os
import numpy as np
import pandas as pd
import re
import time
import helper
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.metrics import accuracy_score

import torch
import torchvision
from torchvision.utils import make_grid
from torchvision import datasets, transforms
from torchvision.io import read_image
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import models
from PIL import Image,ImageDraw,ImageFont
from torch.optim import lr_scheduler
# import splitfolders

In [2]:
os.chdir('/content/drive/MyDrive')

In [ ]:
# input_folder = 'weather_dataset/'

# splitfolders.ratio(input_folder,
#                    output="output",
#                    seed=42,
#                    ratio=(.6, .2, .2),
#                    group_prefix=None)

In [3]:
os.listdir('output')

['train', 'val', 'test']

In [4]:
data_transforms={"train":transforms.Compose([transforms.RandomResizedCrop(224),
                    transforms.RandomHorizontalFlip(),
                    transforms.ToTensor(),
                    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
                    ]),
'val':transforms.Compose([transforms.Resize(256),
                    transforms.CenterCrop(224),
                    transforms.ToTensor(),
                    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
                    ]),
'test':transforms.Compose([transforms.Resize(256),
                    transforms.CenterCrop(224),
                    transforms.ToTensor(),
                    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
                    ])}

image_datasets = {x: datasets.ImageFolder(os.path.join('output', x),data_transforms[x])for x in ['train', 'val','test']}
dataloader={x:torch.utils.data.DataLoader(image_datasets[x],batch_size=8,shuffle=True,num_workers=2) for x in ['train', 'val','test'] }

dataset_size={x :len(image_datasets[x]) for x in ['train', 'val','test']}
class_names = image_datasets['train'].classes


print('class names {}'.format(len(dataloader['train'])))
print('{} batches in training'.format(len(dataloader['train'])))
print('{} batches in validation'.format(len(dataloader['val'])))
print('{} batches in test'.format(len(dataloader['test'])))
print('{} training images'.format(dataset_size['train']))
print('{} validation images'.format(dataset_size['val']))
print('{} test images'.format(dataset_size['test']))

class names 524
524 batches in training
175 batches in validation
176 batches in test
4187 training images
1394 validation images
1404 test images


In [5]:
# model=torchvision.models.resnet34(pretrained=True)
# model1=torchvision.models.efficientnet_v2_l(pretrained=True)
model=torchvision.models.efficientnet_b1(pretrained=True)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B1_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B1_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/efficientnet_b1_rwightman-bac287d4.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b1_rwightman-bac287d4.pth
100%|██████████| 30.1M/30.1M [00:00<00:00, 68.9MB/s]


In [6]:
for param in model.parameters():
  param.requires_grad=False

num_ftrs=model.classifier[-1].in_features
model.classifier[-1]=nn.Linear(num_ftrs,11)

criterion=nn.CrossEntropyLoss()
optimizer=optim.SGD(model.parameters(),lr=0.003,momentum=0.9)
exp_lr_scheduler=lr_scheduler.StepLR(optimizer,step_size=7,gamma=0.1)

In [7]:
result=[]
num_epochs = 50
train_losses, train_accuracies = [], []
val_losses, val_accuracies = [], []

checkpoint_file = "model_checkpoint1.pth"

if os.path.exists(checkpoint_file):
    checkpoint = torch.load(checkpoint_file)
    model.load_state_dict(checkpoint["model_state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
    start_epoch = checkpoint["epoch"]
    print(f"Resuming training from epoch {start_epoch}")
else:
    start_epoch = 0

for epoch in range(start_epoch,num_epochs):
  model.train()
  running_loss = 0.0
  running_corrects = 0
  for inputs, labels in dataloader['train']:
    optimizer.zero_grad()
    outputs = model(inputs)
    _, preds = torch.max(outputs, 1)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    running_loss += loss.item() * inputs.size(0)
    running_corrects += torch.sum(preds == labels.data)

  epoch_loss = running_loss / dataset_size['train']
  epoch_acc = running_corrects.double() / dataset_size['train']
  train_losses.append(epoch_loss)
  train_accuracies.append(epoch_acc)

  model.eval()
  with torch.no_grad():
    running_loss = 0.0
    running_corrects = 0

    for inputs, labels in dataloader['val']:
      outputs = model(inputs)
      _, preds = torch.max(outputs, 1)
      loss = criterion(outputs, labels)

      running_loss += loss.item() * inputs.size(0)
      running_corrects += torch.sum(preds == labels.data)

  epoch_loss = running_loss / dataset_size['val']
  epoch_acc = running_corrects.double() / dataset_size['val']
  val_losses.append(epoch_loss)
  val_accuracies.append(epoch_acc)

  print(f'Epoch {epoch+1}/{num_epochs} - Train Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}  Val Loss: {val_losses[-1]:.4f} Acc: {val_accuracies[-1]:.4f}')

  checkpoint = {
    "epoch": epoch + 1,  # Save the next epoch number
    "model_state_dict": model.state_dict(),
    "optimizer_state_dict": optimizer.state_dict(),}
  torch.save(checkpoint, "model_checkpoint1.pth")

result.append({
    'model': 'vgg16',
    'train_loss': train_losses,
    'train_accuracy': train_accuracies,
    'val_loss': val_losses,
    'val_accuracy': val_accuracies,
})



Resuming training from epoch 45
Epoch 46/50 - Train Loss: 0.6857 Acc: 0.7812  Val Loss: 0.6857 Acc: 0.7812
Epoch 47/50 - Train Loss: 0.6814 Acc: 0.7740  Val Loss: 0.6814 Acc: 0.7740
Epoch 48/50 - Train Loss: 0.6565 Acc: 0.7683  Val Loss: 0.6565 Acc: 0.7683
Epoch 49/50 - Train Loss: 0.7178 Acc: 0.7654  Val Loss: 0.7178 Acc: 0.7654
Epoch 50/50 - Train Loss: 0.6712 Acc: 0.7805  Val Loss: 0.6712 Acc: 0.7805


In [8]:
model.eval()

with torch.no_grad():
  correct=0
  total=0
  total_loss=0
  for images,labels in dataloader['test']:
    output=model(images)
    _,predicted=torch.max(output.data,1)
    total+=labels.size(0)
    correct+=(predicted==labels).sum().item()

    loss=criterion(output,labels)
    total_loss+=loss.item()*labels.size(0)
    avg_loss=total_loss/total

  print('test Accuracy {}' .format(100 * correct/ total))
  print('Validation loss {}'.format(avg_loss))
  print(correct,total)


test Accuracy 81.41025641025641
Validation loss 0.5995633969364682
1143 1404


In [ ]:
plt.plot(range(1,num_epochs+1),val_losses)

In [ ]:
plt.plot(range(1,num_epochs+1),train_losses)

In [ ]:
plt.plot(range(1,num_epochs+1),train_accuracies)

In [ ]:
plt.plot(range(1,num_epochs+1),val_accuracies)